In [1]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
import re
from PIL import Image
import requests
import random
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.corpus import stopwords

In [2]:
books=pd.read_csv('C:\\Users\\akram\\Desktop\\book_recommendation\\backend\data\\Books.csv')
books.head(3)

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...


In [3]:
ratings=pd.read_csv("C:\\Users\\akram\\Desktop\\book_recommendation\\backend\data\\Ratings.csv")
ratings.tail(3)

,User-ID,ISBN,Book-Rating
1149781,16795,0393045218,4
1149782,300000,0439139600,10
1149783,16795,0316973742,7


In [4]:
users=pd.read_csv("C:\\Users\\akram\\Desktop\\book_recommendation\\backend\data\\Users.csv")
users.tail(3)

,User-ID,Location,Age
278859,300100,alger,18.0
278860,300103,Blida,13.0
278861,300200,Mtl,18.0


In [5]:
print("Books Shape: " ,books.shape )
print("Ratings Shape: " ,ratings.shape )
print("Users Shape: " ,users.shape )

Books Shape:  (271360, 8)
Ratings Shape:  (1149784, 3)
Users Shape:  (278862, 3)


In [6]:
books_data = books.merge(ratings, on="ISBN")
books_data.dropna(inplace=True)
books_data = books_data[books_data['Book-Rating'] > 0]
books_data['Book-Title'] = books_data['Book-Title'].apply(lambda x: re.sub("[\W_]+", " ", x).strip())
books_data.head(10)

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L,User-ID,Book-Rating
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,8,5
3,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,11676,8
5,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,67544,8
8,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,116866,9
9,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,123629,9
11,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,200273,8
12,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,210926,9
13,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,219008,7
14,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,263325,6
16,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,2954,8


In [7]:
import pandas as pd
# Compter le nombre de notes par utilisateur et trier par ordre décroissant
ratings_count = ratings.groupby('User-ID').size().sort_values(ascending=False)
# Sélectionner les 50 utilisateurs les plus actifs
top_50_users = ratings_count.head(50)
# Créer un DataFrame avec les informations des 50 utilisateurs
top_50_users_df = pd.DataFrame({
    'User-ID': top_50_users.index,
    'Rating-Count': top_50_users.values
})
filtered_users_df = users[users['User-ID'].isin(top_50_users_df['User-ID'])]
filtered_users_df.head()

,User-ID,Location,Age
11600,11601,"brooklyn, new york, usa",NaN
11675,11676,"n/a, n/a, n/a",NaN
16794,16795,"mechanicsville, maryland, usa",47.0
23767,23768,"st. louis, missouri, usa",45.0
26543,26544,"woodbridge, virginia, usa",37.0


In [8]:
def get_top_n_ratings_for_user(books_data, user_id, n=5):
    
    # Filter data for the specific user
    user_data = books_data[books_data['User-ID'] == user_id]
    
    # Sort the data by 'Book-Rating' in descending order and slice the top n entries
    top_n_ratings = user_data.sort_values(by='Book-Rating', ascending=False).head(n)
    
    return top_n_ratings[['User-ID', 'Book-Title', 'Book-Rating', 'ISBN']]
user_id = 300000
get_top_n_ratings_for_user(books_data, user_id, n=10).head(50)

,User-ID,Book-Title,Book-Rating,ISBN
213781,300000,Harry Potter and the Goblet of Fire Book 4,10,0439139600
96169,300000,Harry Potter and the Sorcerer s Stone Harry Po...,9,059035342X
